In [ ]:
import os
import numpy as np
import time

import meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
import meshcat.transformations as tf

import pinocchio as pin

server_args = []
package_dirs = './'
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

viewer = meshcat.Visualizer(zmq_url=zmq_url)

In [ ]:
viewer['dinosaur1'].set_object(meshcat.geometry.StlMeshGeometry.from_file(os.path.join(package_dirs, "dinosaur.stl")),\
                              meshcat.geometry.MeshLambertMaterial(
                              color=0xff22dd))
viewer['dinosaur2'].set_object(meshcat.geometry.StlMeshGeometry.from_file(os.path.join(package_dirs, "dinosaur.stl")),\
                              meshcat.geometry.MeshLambertMaterial(
                              color=0xffdd22))

In [ ]:
dt = 1/120.
dat = np.loadtxt(open('data.csv', "rb"), delimiter=",", dtype=float, skiprows=1)
N = dat.shape[0]
quat_dat = [list(dat[i, 3:]) for i in range(N)]
quat = [pin.Quaternion(np.matrix(quat_dat[i]).T).normalized() for i in range(N)]

In [ ]:
for i in range(N):
    rot_tf = np.identity(4)
    rot_tf[:3,:3] = quat[i].toRotationMatrix().dot(pin.rpy.rotate('x', np.pi/2)).dot(pin.rpy.rotate('z', np.pi))
    tz = tf.translation_matrix(dat[i, :3])
    viewer['dinosaur2'].set_transform(tz.dot(rot_tf))
    time.sleep(dt/2)